In [1]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "..", "src"))
from schema_builder import create_tables
from data_loader import load_data_to_db

create_tables()
load_data_to_db("../data/processed/reviews_final.csv")


Tables created successfully.
Loading data from ../data/processed/reviews_final.csv into PostgreSQL database...
Data successfully loaded into PostgreSQL!


In [3]:


from db_connection import get_engine
engine = get_engine()
from sqlalchemy import text
import pandas as pd

query = """
SELECT b.bank_name, COUNT(*) AS review_count
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name
ORDER BY review_count DESC;
"""

with engine.begin() as conn:
    result = conn.execute(text(query))
    df_res = pd.DataFrame(result.fetchall(), columns=result.keys())

df_res


,bank_name,review_count
0,Bank of Abyssinia,319
1,Commercial Bank of Ethiopia,307
2,Dashen Bank,295


In [7]:
sql = """
SELECT 
    b.bank_name,
    r.vader_sentiment,
    COUNT(*) as total
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name, r.vader_sentiment
ORDER BY b.bank_name, total DESC;
"""

with engine.begin() as conn:
    df_sent = pd.read_sql(text(sql), conn)

df_sent



,bank_name,vader_sentiment,total
0,Bank of Abyssinia,positive,127
1,Bank of Abyssinia,neutral,115
2,Bank of Abyssinia,negative,77
3,Commercial Bank of Ethiopia,positive,154
4,Commercial Bank of Ethiopia,neutral,124
5,Commercial Bank of Ethiopia,negative,29
6,Dashen Bank,positive,173
7,Dashen Bank,neutral,73
8,Dashen Bank,negative,49


In [8]:
sql = """
SELECT 
    b.bank_name,
    r.theme,
    COUNT(*) as total
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name, r.theme
ORDER BY b.bank_name, total DESC;
"""

df_theme = pd.read_sql(text(sql), engine)
df_theme


,bank_name,theme,total
0,Bank of Abyssinia,Other,281
1,Bank of Abyssinia,Performance,23
2,Bank of Abyssinia,GenericIssues,7
3,Bank of Abyssinia,Login,3
4,Bank of Abyssinia,Network,3
5,Bank of Abyssinia,Transactions,2
6,Commercial Bank of Ethiopia,Other,270
7,Commercial Bank of Ethiopia,Performance,15
8,Commercial Bank of Ethiopia,Transactions,6
9,Commercial Bank of Ethiopia,GenericIssues,6


In [9]:
sql = """
SELECT 
    theme,
    AVG(rating) AS avg_rating,
    COUNT(*) as review_count
FROM reviews
GROUP BY theme
ORDER BY avg_rating DESC;
"""
df_rating_theme = pd.read_sql(text(sql), engine)
df_rating_theme


,theme,avg_rating,review_count
0,UI/UX,4.250000,4
1,Other,3.743590,819
2,Network,3.500000,10
3,Login,2.777778,9
4,Transactions,2.727273,11
5,Performance,2.300000,50
6,GenericIssues,2.166667,18


In [10]:
sql = """
SELECT 
    b.bank_name,
    AVG(vader_compound) as avg_sentiment
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
GROUP BY b.bank_name
ORDER BY avg_sentiment DESC;
"""
df_sent_avg = pd.read_sql(text(sql), engine)
df_sent_avg


,bank_name,avg_sentiment
0,Dashen Bank,0.280528
1,Commercial Bank of Ethiopia,0.231582
2,Bank of Abyssinia,0.079696
